# Db2 External Tables with Cloud Storage


##### This notebook runs the queries shown in my db2Dean article

See:  http://www.db2dean.com/Previous/ExternalTablesCloud.html



### download the db2.ipynb notebook from Git to allow the Db2 Magic Commands

In [1]:
%run db2.ipynb

Pixiedust database opened successfully


DB2 Extensions Loaded.


Connect to your db2 11.5 or above database.  

In [2]:
%sql CONNECT TO sample USER db2inst1 USING ? HOST 192.168.0.17 PORT 50000

Password [password]: ········
Connection successful.


In [4]:
%sql CONNECT RESET

Connection closed.


##### Create the external  table for the hospital readmission file.
Make sure to change the cloud credentials, file name and bucket name

In [18]:
%%sql -a 
DROP TABLE dean.HOSPITAL_READMISSION_EXT;
CREATE EXTERNAL TABLE DEAN.HOSPITAL_READMISSION_EXT
   (HOSPITAL_NAME VARCHAR(200)  ,               
    PROVIDER_NUMBER INTEGER  ,              
    STATE VARCHAR(4)  ,                        
    MEASURE_NAME VARCHAR(30)  ,                
    NUMBER_OF_DISCHARGES INTEGER,
    FOOTNOTE VARCHAR(4)  ,                     
    EXCESS_READMISSION_RATIO DECIMAL(20,4),
    PREDICTED_READMISSION_RATE DECIMAL(20,4),
    EXPECTED_READMISSION_RATE DECIMAL(20,4),
    NUMBER_OF_READMISSIONS INTEGER  ,      
    START_DATE DATE  ,                 
    END_DATE DATE)          
USING (dataobject 'HOSPITAL_READMISSION_LOAD.csv'
          s3('s3.us-south.cloud-object-storage.appdomain.cloud',
             '99b77f8a0c6f47f09658dd1cf0187deb',
             '3522e1666e319b6b42aed090294a0abf47611e2b8b55dc91',
             'external-table-cloud-bucket')
          maxerrors 100000 
          DELIMITER ','  
          DATEDELIM '-'
          Y2BASE 2000
          DATESTYLE 'DMONY2'
          -- MAXROWS 200    
          STRING_DELIMITER DOUBLE 
          SKIPROWS 1 
          FILLRECORD True
      )
;

Command completed.


Count the rows in the external table.  This will only show valid records.  
Running this or any query will cause the .log and .bad files to be written.

In [10]:
%sql select count(*) from dean.HOSPITAL_READMISSION_EXT 

,1
0,11638


Create a column organized readmissions table 

In [21]:
%%sql -a
DROP TABLE DEAN.HOSPITAL_READMISSION;
CREATE  TABLE dean.HOSPITAL_READMISSION
   (HOSPITAL_NAME VARCHAR(200)  ,               
    PROVIDER_NUMBER INTEGER  ,              
    MEASURE_NAME VARCHAR(30)  ,                
    PREDICTED_READMISSION_RATE DECIMAL(20,4),
    START_DATE DATE)                 
 ORGANIZE BY COLUMN
;


Command completed.


Copy the valid rows from the external table to the column organized table using insert with select.

In [ ]:
%%sql -a
insert into dean.hospital_readmission
   (HOSPITAL_NAME  ,               
    PROVIDER_NUMBER  ,
    MEASURE_NAME   , 
    PREDICTED_READMISSION_RATE ,
    START_DATE)     
select     
    HOSPITAL_NAME  ,               
    PROVIDER_NUMBER ,             
    MEASURE_NAME ,
    PREDICTED_READMISSION_RATE ,
    START_DATE
    FROM dean.hospital_readmission_ext   
;
 


In [ ]:
%sql call sysproc.admin_cmd('describe table dean.hospital_readmission_ext')

Copy the valid rows from the external table to the column organized table using the load utility.

In [23]:
%%sql -a
call sysproc.admin_cmd('load from (select HOSPITAL_NAME,               
                                        PROVIDER_NUMBER ,             
                                        MEASURE_NAME ,
                                        CAST(PREDICTED_READMISSION_RATE AS DECIMAL(20,4)) ,
                                        START_DATE
                             from dean.hospital_readmission_ext order by start_date) 
                       of cursor messages on server 
                       replace resetdictionary 
                       into dean.hospital_readmission(HOSPITAL_NAME  ,               
                                                    PROVIDER_NUMBER  ,
                                                    MEASURE_NAME   , 
                                                    PREDICTED_READMISSION_RATE ,
                                                    START_DATE)    
                      ');

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [19]:
%%sql -a
SELECT SQLCODE, MSG FROM TABLE(SYSPROC.ADMIN_GET_MSGS('2073449332_1928906487_DB2INST1')) AS MSG 

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [3]:
%sql select count(*) from dean.hospital_readmission 

,1
0,11638


## Switching gears to work with JSON data

Create an external table to hold JSON documnets, putting one document in each row

In [43]:
%%sql -a 
DROP TABLE dean.energy_json_ext;
CREATE EXTERNAL TABLE dean.energy_json_ext
  (JSON_FIELD clob(2000))
USING (dataobject 'catenergy.json'
          s3('s3.us-south.cloud-object-storage.appdomain.cloud',
             '99b77f8a0c6f47f09658dd1cf0187deb',
             '3522e1666e319b6b42aed090294a0abf47611e2b8b55dc91',
             'external-table-cloud-bucket')
          maxerrors 100000 
          DELIMITER '|'  
      );

Command completed.


Select some of the JSON data nicely formatted for easy viewing

In [57]:
%%sql -j
select json_field from dean.energy_json_ext fetch first 5 rows only

[{'energy': {'timeUnit': 'HOUR',
   'unit': 'Wh',
   'measuredBy': 'INVERTER',
   'values': [{'date': '2017-01-27 00:00:00', 'value': None},
    {'date': '2017-01-27 01:00:00', 'value': None},
    {'date': '2017-01-27 02:00:00', 'value': None},
    {'date': '2017-01-27 03:00:00', 'value': None},
    {'date': '2017-01-27 04:00:00', 'value': None},
    {'date': '2017-01-27 05:00:00', 'value': 0.0},
    {'date': '2017-01-27 06:00:00', 'value': None},
    {'date': '2017-01-27 07:00:00', 'value': None},
    {'date': '2017-01-27 08:00:00', 'value': 5.0},
    {'date': '2017-01-27 09:00:00', 'value': 35.0},
    {'date': '2017-01-27 10:00:00', 'value': 176.0},
    {'date': '2017-01-27 11:00:00', 'value': 404.0},
    {'date': '2017-01-27 12:00:00', 'value': 881.0},
    {'date': '2017-01-27 13:00:00', 'value': 1293.0},
    {'date': '2017-01-27 14:00:00', 'value': 2069.0},
    {'date': '2017-01-27 15:00:00', 'value': 2155.0},
    {'date': '2017-01-27 16:00:00', 'value': 650.0},
    {'date': '2017-

### Create a function to validate JSON records
Queries on the table will fail if even one bad document if you don't use this

In [50]:
%%sql -d
CREATE OR REPLACE FUNCTION DB2INST1.CHECK_JSON(JSON CLOB)
      RETURNS INTEGER
      CONTAINS SQL LANGUAGE SQL DETERMINISTIC NO EXTERNAL ACTION
  BEGIN
    DECLARE RC BOOLEAN;
    DECLARE EXIT HANDLER FOR SQLEXCEPTION RETURN(FALSE);
    SET RC = JSON_EXISTS(JSON,'$' ERROR ON ERROR);
    RETURN(TRUE);
END

Command completed.


Create a table with some columns to hold JSON fields

In [55]:
%%sql -a 
DROP TABLE dean.energy_json;
CREATE TABLE dean.energy_json
    (SEQ INT NOT NULL GENERATED ALWAYS AS IDENTITY,
     READING_TS TIMESTAMP NOT NULL WITH DEFAULT,
     ENERGY_UNIT VARCHAR(20),
     NOON_THROUGH_2_TOTAL DECIMAL(10,1),
     JSON_FIELD blob(2000));


Command completed.


Copy data into the table extracting some JSON fields, and getting the fill JSON document converted to BSON

In [53]:
 %%sql
insert into dean.energy_json
   (READING_TS,
    ENERGY_UNIT,
    NOON_THROUGH_2_TOTAL,
    JSON_FIELD)     
select JSON_VALUE(json_field, '$.energy.values[0].date' RETURNING TIMESTAMP),
       CASE WHEN JSON_VALUE(json_field, '$.energy.unit' RETURNING VARCHAR(20)) = 'Wh' THEN 'WATT HOURS' END,
       JSON_VALUE(json_field, '$.energy.values[12].value' RETURNING Decimal(10,1)) 
            + JSON_VALUE(json_field, '$.energy.values[13].value' RETURNING Decimal(10,1))
            + JSON_VALUE(json_field, '$.energy.values[14].value' RETURNING Decimal(10,1)),
       JSON_TO_BSON(JSON_FIELD) 
    FROM dean.energy_json_ext   
   WHERE DB2INST1.CHECK_JSON(JSON_FIELD)
;
    
 
    

Command completed.


In [54]:
%sql select reading_ts, JSON_VALUE(json_field, '$.energy.unit' RETURNING VARCHAR(20))  from dean.energy_json

,READING_TS,2
0,2017-01-27,Wh
1,2017-01-29,Wh
2,2017-01-30,Wh
3,2017-01-31,Wh
4,2017-02-01,Wh
...,...,...
561,2019-12-14,Wh
562,2019-12-15,Wh
563,2019-12-16,Wh
564,2019-12-17,Wh


## Options
Both forms of the **`%sql`** command have options that can be used to change the behavior of the code. For both forms of the command (**`%sql`**, **`%%sql`**), the options must be on the same line as the command:
<pre>
%sql -t ...
%%sql -t
</pre>

The only difference is that the **`%sql`** command can have SQL following the parameters, while the **`%%sql`** requires the SQL to be placed on subsequent lines.

There are a number of parameters that you can specify as part of the %sql statement. 

* **`-d`** - Use alternative delimiter
* **`-t`** - Time the statement execution
* **`-n`** - Run all statements as commands (no answer sets)
* **`-s`** - Run all statements as SQL
* **`-q`** - Suppress messages 
* **`-j`** - JSON formatting of a column
* **`-a`** - Show all output
* **`-pb`** - Bar chart of results
* **`-pp`** - Pie chart of results 
* **`-pl`** - Line chart of results
* **`-i`** - Interactive results and plotting
* **`-sampledata`** Load the database with the sample EMPLOYEE and DEPARTMENT tables
* **`-r`** - Return the results into a variable (list of rows)

Multiple parameters are allowed on a command line. Each option should be separated by a space:
<pre>
%sql -a -j ...
</pre>

A SELECT statement will return the results as a dataframe and display the results as a table in the notebook. If you use the assignment statement, the dataframe will be placed into the variable and the results will not be displayed:
<pre>
r = %sql SELECT * FROM EMPLOYEE
</pre>

The sections below will explain the options in more detail.